# In-Context Learning
* Be careful when doing this, as the cost is very high.

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@develop

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.47


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [4]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import ICLClassifier

In [29]:
#seed = 3655
#seed = 3656
seed = 3657
train_data = 300

file_name = f'pseudodata_{seed}_{train_data}_train.csv'

i = 200

df = pd.read_csv(f'../data/pseudodata/{file_name}').head(i)
x_train = df.drop('target', axis=1)
y_train = df['target']
print(len(x_train))

200


In [30]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iclm = ICLClassifier(llm_model = llm_model)

### Make prompt

In [31]:
icl_prompt = iclm.fit(x_train, y_train)

Number of input tokens:5693


In [32]:
print(icl_prompt)


        Predict the probability value by observing all of the following conditions.
        ・Output must be numeric only.
        ・Do not output any text.
        ・Predicting probability values as finely as possible increases overall accuracy.
        ・Please refer to the following data to estimate the probability that "target" will be 1. The rightmost column with a value of 0 or 1 is 'target'.
        ------------------
        0.71,-1.08,-0.83,-1.21,1.0
-0.25,0.49,0.32,0.51,0.0
0.22,-0.0,-0.16,-0.12,1.0
-0.64,1.27,0.83,1.31,0.0
2.04,2.13,-0.86,0.47,1.0
-1.41,-1.27,0.65,-0.18,0.0
0.3,-1.15,-0.55,-1.03,1.0
-2.7,1.35,2.36,2.52,0.0
0.42,0.22,-0.24,-0.07,1.0
-0.62,1.21,0.81,1.26,0.0
0.98,1.09,-0.39,0.28,1.0
-0.76,-0.97,0.27,-0.31,0.0
0.16,0.2,-0.06,0.06,1.0
-1.1,1.05,1.11,1.4,0.0
1.55,-1.17,-1.47,-1.74,1.0
-0.67,-0.59,0.31,-0.07,0.0
0.03,-0.23,-0.09,-0.19,1.0
-0.72,-0.69,0.32,-0.12,0.0
1.39,-0.34,-1.11,-1.03,1.0
-0.72,0.63,0.71,0.87,0.0
1.57,1.52,-0.69,0.28,1.0
-0.68,-0.65,0.31,-0.11,0.0

### Prediction

In [33]:
file_name = f'pseudodata_{seed}_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

In [34]:
y_proba = iclm.predict(x_test)

y_pred = (y_proba > 0.5).astype(int)

output = pd.DataFrame(y_proba, columns=['pred'])
data_name = 'pseudodata'
output.to_csv(f'../data/icl_output/pred_{data_name}_{seed}.csv')

 46%|████▌     | 138/300 [04:00<04:24,  1.63s/it]Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-rKmEAn8qrwuz778vK8jNCcB5 on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 04 Jul 2023 06:09:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate,

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(accuracy, 4)}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {round(precision, 4)}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {round(recall, 4)}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {round(f1, 4)}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {round(roc_auc, 4)}')

Accuracy: 0.8967
Precision: 0.8861
Recall: 0.915
F1 score: 0.9003
ROC-AUC: 0.9486


In [8]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_test.csv'

df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

prefix = 1
y_proba = pd.read_csv(f'../data/icl_output/pred.csv')
y_proba = y_proba.drop('Unnamed: 0', axis=1)
y_pred = (y_proba > 0.5).astype(int)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

ValueError: Found input variables with inconsistent numbers of samples: [700, 300]